# GENDER RECOGNITION BY VOICE
###   SIMAN PARVEEN(521)
###   SAWAN AICH(553)
###   RAHUL CHATTERJEE(545)
####   GUIDED BY KAUSHIK GOSWAMI

## Abstract:
If there is something that differentiates human beings from other animals, it is our ability to use
language to speak and communicate with each other.Now, distinguishing a male voice from a female voice
seems to be an easy task for a human being.But it becomes difficult when a computer has to identify it
whether the voice is of male or female. A human has that natural capability of idenitifying the 
difference but when it comes to computer, we need to teach it by providing inputs, methodology or 
different training data and make it learn. In this project, we focus on training the computer which can 
identify a person’s gender if a voice is given as the input.

## Import the required libraries

In [1]:
#LIBRARIES FOR APP
from flask import Flask,render_template,url_for,request,redirect
from flask_uploads import UploadSet,AUDIO,configure_uploads,extension,UploadConfiguration 
from werkzeug import secure_filename
from flask_ngrok import run_with_ngrok

#LIBRARIES FOR READING DATASET,PREPROCESSING,SPLIT,FEATURE EXTRACTION
import pickle
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import librosa
from scipy.io import wavfile
from scipy.stats import stats
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 

## Read the Csv File Created
<a href="http://localhost:8888/notebooks/Desktop/Final%20Year%20Project/Dataset_Creation.ipynb">Click on this For Detailed Knowledge of Feature Extraction And Dataset Creation</a>


<a href="http://localhost:8888/notebooks/Desktop/Final%20Year%20Project/DatasetCreation2.ipynb">Click Here to know How we have added more audios to Dataset</a>


In [2]:
# The csv file is read and stored in 'df' 
df=pd.read_csv("C:\\Users\\SAGAR\\Desktop\\Final Year Project\\csv_dataset\\dataset.csv")
# Counted the number of male and female in the training dataset
df['LABEL'].value_counts()

m    12595
f     9603
Name: LABEL, dtype: int64

## Splitting The Data in the Dataset

In [3]:
train,test=train_test_split(df,test_size=0.0)# the whole dataset is actually used for training
train.index=range(len(train))# to set the index
x_train=train.iloc[:,:-1]#to take all the columns except the 'LABEL' column
y_train=train[["LABEL"]]#'LABEL' column is considered a separate dataframe

## Preprocessing the data using
   -- Label Encoder(Female-1,Male-0)
   
   -- MinMax Scaler(To scale the value between 0 and 1)

In [4]:
le=preprocessing.LabelEncoder() #this preprocessing technique is used to scale the labels-Female and Male 
y_train=le.fit_transform(y_train.values.ravel())# the values are being fitted

In [ ]:
scaler=preprocessing.MinMaxScaler()#this preprocessing technique is used to scale the values between 0 and 1 
x_train[:]=scaler.fit_transform(x_train.astype(float))

## Function to extract the features from the given audio

### We have to  extract the following features from the given audio-
       -- Dominating Frequency(meanfreq)
       -- Min Dominating Frequency(low)
       -- Max Dominating Frequency(peak)
       -- Standard Deviation(std)
       -- Kurtosis(kurt)
       -- Skewness(skew)
       -- Mode(mode)
       -- Median(median)
       -- First Quartile(Q25)
       -- Third Quartile(Q75)
       -- Inter Quartile Range(iqr)
       -- Spectral Centroid(sp_cent)
       -- Spectral Flatness(sfm)
       -- Spectral Roll-off(sp_rolloff)
       -- Spectral BandWidth(sp_bandwidth)

In [ ]:
def get_features(audio):
        mean1=[]
        skew1=[]
        kurtosis1=[]
        median1=[]
        mode1=[]
        std1=[]
        low1=[]
        peak1=[]
        q751=[]
        q251=[]
        iqr1=[]
        sp_cent_1=[]
        sfm_1=[]
        sp_rolloff_1=[]
        sp_bandwidth_1=[]
        rate,data=wavfile.read(audio)#Reading the audio wav file 
        step=int(rate/5)#the size of the window frequency
        freqArray2=[]
        freqArray1=[]
        sp_cent=[]
        #Find out the Dominating Frequency Using Fourier
        for j in range(1,len(data),step):
            fourier=np.fft.fft(data[j:j+step])
            freqs=np.fft.fftfreq(len(fourier))#freqs tells you the frequencies associated with the coefficients
            imax=np.argmax(np.abs(fourier))#returns the max coeffient
            freq=freqs[imax]#returns the frequency assciated with max coefficient
            freq_in_hz=abs(freq*rate)
            freqArray1.append(freq_in_hz)
            x=[f for f in freqArray1 if 20<f<300 and not 46<f<66]#so that only human voice is considered
        freqArray2.append(x)
        freqArray=[item for sublist in freqArray2 for item in sublist]
        
        x, sr=librosa.load(audio)#loading the audio
        spectral_centroids = librosa.feature.spectral_centroid(x, sr=sr)[0]#to find the spectral centroid of the audio 
        flatness=librosa.feature.spectral_flatness(x)#to find the spectral flatness of the audio
        rolloff = librosa.feature.spectral_rolloff(x,sr=sr)#to find the spectral rolloff of the audio
        bandwidth=librosa.feature.spectral_bandwidth(x,sr=sr)#to find the spectral bandwidth of the audio
        
        sfm=np.mean(flatness)#to find the mean of all the spectral flatness obtained of an audio
        sp_cent=np.mean(spectral_centroids)#to find the mean of all the spectral centroid obtained of an audio
        sp_rolloff=np.mean(rolloff)#to find the mean of all the spectral rolloff obtained of an audio
        sp_bandwidth=np.mean(bandwidth)#to find the mean of all the spectral bandwidth obtained of an audio
        
        nobs,minmax,mean,variance,skew,kurtosis=stats.describe(freqArray)#to find out the dominating frquency,skewness and kurtosis of the audio
        mode=stats.mode(freqArray).mode[0]#to find out the mode of the frequencies of the audio
        std=np.std(freqArray)#to find out the standard deviation 
        low,peak=minmax#to find out the min and max dominating frequency
        q75,q25=np.percentile(freqArray, [75 ,25])#to find out first quartile and second quartile
        iqr=q75-q25#to find out the interquartile range
        median=np.median(freqArray)#to find out the median

        #Here the array for each column in the dataset is created by appending the value obtained for above
        #for the given audio
        mean1.append(mean/1000)
        skew1.append(skew)
        kurtosis1.append(kurtosis)
        median1.append(median/1000)
        mode1.append(mode/1000)
        std1.append(std/1000)
        low1.append(low/1000)
        peak1.append(peak/1000)
        q751.append(q75/1000)
        q251.append(q25/1000)
        iqr1.append(iqr/1000)
        sp_cent_1.append(sp_cent)
        sfm_1.append(sfm)
        sp_rolloff_1.append(sp_rolloff)
        sp_bandwidth_1.append(sp_bandwidth)
        
        d=({'meanfreq':mean1,'sd':std1,'median':median1,'Q25':q251,'Q75':q751,'iqr':iqr1,'skew':skew1,'kurt':kurtosis1,'mode':mode1,'low':low1,'peak':peak1,'centroid':sp_cent_1,'flatness':sfm_1,'spectral rolloff':sp_rolloff_1,'spectral bandwidth':sp_bandwidth_1})
        df_audio=pd.DataFrame(data=d)
        return df_audio

### Function to Predict the gender of the Person speaking in the audio

In [ ]:
def prediction(x_test, clf_object):
    LABEL=['F','M']#the labels(F-Female,M-Male)
    pred=[] 
    y_pred = clf_object.predict(x_test) #Prediction
    y=int(y_pred)
    pred.append(LABEL[y])   
    return pred #returns either the gender is 'F' or 'M'(M-Male,F-Female)

# Load the Ensemble Model
### Ensemble Model consists of the following three models-
       -- Gradient  Boosting
       -- Support Vector Machine
       -- Random Forest
#### To know how we have created and saved the model <a href="http://localhost:8888/notebooks/Desktop/Final%20Year%20Project/model_creation.ipynb">Click Here</a>       

In [ ]:
model=pickle.load(open('ensemble_model1.sav','rb'))  #loading the ensemble model
print("\n The Model Is Ready")


 The Model Is Ready


# FLASK APP

### Here the GUI is created using flask(using python) in Back END
### The Front End is created using HTML and CSS
### Details Present Here:
    clear_all_Uploadedfile()-Function to delete all the audio files present in the back end folder.
    allow_audio()-Function to specify which type of audio file is allowed here(.WAV).
    The following Functions specfies what the functions returns:
        index()-Function to open the Home page of the app.
        predict()-Function to play the audio.
        gender()-Function to display the gender of the person speaking in the audio.</b>

In [ ]:
#Here we delete the previous uploaded files because while getting the audio from the audio folder
#in the static folder,always the first audio is considered.
#So to overcome this problem,We are deleting the previous uploaded files

def clear_all_Uploadedfile():
    path_to_upld_folder=os.path.join(os.getcwd(),'static/audio')#to get the path of the folder where the audios are saved
    list_of_auds=os.listdir(path_to_upld_folder)#to get the audios present
    for aud in list_of_auds:
        path_to_aud=os.path.join(path_to_upld_folder,aud)
        os.remove(path_to_aud)#delete the existing audios
        print(f"\nDeleted {aud} \n")#print the audio name deleted
    print("All Previous Uploaded Audios Deleted")

  
app=Flask(__name__)#the app is initialised
app.config['AUDIO_EXTENSION']=['wav']#to specify the type of audio accepted(to use later in allow_audio()function)
#run_with_ngrok(app)#so that anyone can test the GUI if the sever is on

#UploadConfiguration(os.path.join(os.getcwd(),'static/audio'),\
                    #allow=app.config['AUDIO_EXTENSION'])

#We have allow_audio() function to check whether the user uploads only .wav audio file
#We have considered only .wav audio file because .wav audio file contains almost all the features of the 
#audio. Conversion to .mp3 to .wav is not considered here because the .mp3 type does not contain all the 
#audio features as .wav.

def allow_audio(filename):
    if not '.' in filename:#if no extension is present
        return False
    ext=filename.rsplit('.',1)[1]#splitting the given url on the basis of '.' and getting the type of file uploaded
    if ext.lower() in app.config['AUDIO_EXTENSION']:
        return True # if we have wav type return that its acceptable
    else:
        return False # else not acceptable

@app.route('/')
def index():
    return render_template('index.html')  # the home page is returned


@app.route('/predict',methods=['POST'])
def predict():
    if request.method=="POST":
        clear_all_Uploadedfile()#delete all the previous uploaded audios
        if request.files and 'audio' in request.files:#if there is any file in the uploaded region
            audio_file = request.files['audio']#audio_file is initialized with the name of the audio file
            if not allow_audio(audio_file.filename):#checks whether audio file is not of .wav type
                print("The audio extension is not allowed")
                return redirect('/')#redirect to the home page(index() function)
            
            audio_file.save(os.path.join(os.getcwd(),'static/audio',\
                                             secure_filename(audio_file.filename)))#if the type is of wav type, 
                                                                                #save the file in the folder 
            print(f'Audio File {audio_file} Saved')
            return render_template('predict.html',audio=audio_file.filename)#after the file is saved,
                            #the file is post at the predict.html so that we can play the audio


@app.route('/gender',methods=['POST','GET'])
def gender():
    if request.method=='POST':#we have used POST here ,to get the audio file from predict.html
        audio_file=request.form['audio']
        path=os.path.join(os.getcwd(),'static\\audio',audio_file)# from the current directory  
                                #goes to static folder and then audio folder and get the whole path
        print(f"\n The Audio file: {path} ")
        df_audio=get_features(path)#the audio path is send to the get_features()function for feature
                                #extraction
        df_audio[:]=scaler.transform(df_audio)#scaled according to the training dataset
        df_test_pred=prediction(df_audio,model)#predict the gender(calls the function prediction())
        if df_test_pred==['F']:
            df_test='Female'#if person speaking is Female
        else:
            df_test='Male'#if person speaking is Male
        print("Result Out")    
        return render_template('gender.html',prediction_text="In the given Audio,the person speaking is {}".format(df_test))
                                      #returns the predict.html and the final result
        

if __name__=='__main__':
    app.run()#runs the flask app

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Mar/2020 13:10:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2020 13:10:46] "GET /static/style1.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2020 13:10:46] "GET /static/_87558504_87558503.jpg HTTP/1.1" 200 -



Deleted SantoshSIR.wav 

All Previous Uploaded Audios Deleted
Audio File <FileStorage: 'Pritam.wav' ('audio/wav')> Saved


127.0.0.1 - - [21/Mar/2020 13:11:06] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2020 13:11:07] "GET /static/audio/Pritam.wav HTTP/1.1" 200 -



 The Audio file: C:\Users\SAGAR\Desktop\Final Year Project\static\audio\Pritam.wav 


...

127.0.0.1 - - [21/Mar/2020 13:14:02] "GET /static/audio/Oishi.wav HTTP/1.1" 200 -



 The Audio file: C:\Users\SAGAR\Desktop\Final Year Project\static\audio\Oishi.wav 


127.0.0.1 - - [21/Mar/2020 13:14:04] "POST /gender HTTP/1.1" 200 -


Result Out


127.0.0.1 - - [21/Mar/2020 13:14:06] "GET / HTTP/1.1" 200 -


# Conclusion:
### After the completion of the project, we came up with the following conclusions:
	--The accuracy of the machine learning models differs when the dataset being used 
    for training is changed.
	--If more acoustic features or properties of the input voice is extracted,more 
    accurate is the result.
    --Deep female voice or shrill male voice may sometimes lead to wrong prediction.
	--Better accuracy is obtained when the ratio of male and female voices of the 
    trained dataset is nearly 1:1.
	--There are major scopes of further betterment of this project, such as separation
    of various audios in the foreground and background of an input audio file,and so on.


# Thank You!!!!